In [1]:
import warnings
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from lightgbm import LGBMClassifier
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
train = pd.read_parquet("train_nontemporal_classification.parquet.gz")
test =  pd.read_parquet("test_nontemporal_classification.parquet.gz")

In [3]:
train.head()

,powiat_voivod,cloud_cover,wind_speed,humidity,precipitation,snow_depth,temperature,voivodship,animal stock - total,area by land - utilised agricultural area,...,"crop production - permanent meadows, area in ha, total","crop production - permanent pastures, area in ha, total",CAQI_level,isUrban,season,holiday_name,is_school_holiday,is_lockdown,is_have_coalplant,is_weekend
DATE,,,,,,,,,,,,,,,,,,,,,
2017-01-01,"powiat aleksandrowski, kujawsko-pomorskie",4.746750,0.0,88.4,1.0,0.0,1.8,kujawsko-pomorskie,44.997895,1.718015,...,4.884211,0.854737,vlow,0,Winter,New Year's Day,0.0,0.0,0.0,1
2017-01-01,"powiat augustowski, podlaskie",8.000000,50.0,95.0,9.0,0.0,0.4,podlaskie,47.547318,4.963793,...,16.300784,2.241712,medium,0,Winter,New Year's Day,0.0,0.0,0.0,1
2017-01-01,"powiat bełchatowski, łódzkie",5.923833,0.0,80.0,0.0,5.0,0.0,łódzkie,43.636364,3.809493,...,7.991736,0.686983,vlow,0,Winter,New Year's Day,0.0,0.0,1.0,1
2017-01-01,"powiat biała podlaska, lubelskie",6.000000,0.0,78.0,0.0,0.0,0.9,lubelskie,21.265306,0.138255,...,8.612245,0.551020,medium,1,Winter,New Year's Day,0.0,0.0,0.0,1
2017-01-01,"powiat białystok, podlaskie",8.000000,0.0,89.0,0.0,0.0,0.4,podlaskie,47.549020,0.305201,...,16.303922,2.245098,low,1,Winter,New Year's Day,0.0,0.0,0.0,1


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 300960 entries, 2017-01-01 to 2021-02-28
Columns: 123 entries, powiat_voivod to is_weekend
dtypes: float64(116), int32(1), int64(1), object(5)
memory usage: 283.6+ MB


In [4]:
train.shape

(300960, 123)

In [5]:
train.isnull().sum()

powiat_voivod        0
cloud_cover          0
wind_speed           0
humidity             0
precipitation        0
                    ..
holiday_name         0
is_school_holiday    0
is_lockdown          0
is_have_coalplant    0
is_weekend           0
Length: 123, dtype: int64

In [6]:
train['CAQI_level'].value_counts()

vlow      134555
low       126762
medium     30737
high        8026
vhigh        880
Name: CAQI_level, dtype: int64

In [6]:
# combine high and vhigh to a single category, as we want to see how it will affect the accuracy of model
train['CAQI_level'] = train['CAQI_level'].replace({'vhigh':'high'})

In [7]:
train['CAQI_level'].value_counts()

vlow      134555
low       126762
medium     30737
high        8906
Name: CAQI_level, dtype: int64

In [8]:
# The ordered of mapping
mapping = {'vlow': 0, 'low': 1, 'medium': 2, 'high': 3}
train['CAQI_level'] = train['CAQI_level'].replace(mapping)

In [10]:
train['CAQI_level'].value_counts()

0    134555
1    126762
2     30737
3      8906
Name: CAQI_level, dtype: int64

In [11]:
train.columns.to_list()

['powiat_voivod',
 'cloud_cover',
 'wind_speed',
 'humidity',
 'precipitation',
 'snow_depth',
 'temperature',
 'voivodship',
 'animal stock - total',
 'area by land - utilised agricultural area',
 'area by land - forests',
 'area by land - residential areas',
 'area by land - industrial lands',
 'area by land - recreational and rest areas',
 'area by land - lands under waters',
 'area by land - wasteland',
 'area by land - other area',
 'emission of particulates - fugitive',
 'emission of particulates - fuel combustion',
 'emission of particulates - cement/lime and refractory materials',
 'emission of particulates - silicon',
 'emission of particulates - chemical fertilizers',
 'emission of particulates - surface-active agents',
 'emission of particulates - carbon and graphite, soot',
 'emission of pollutant gases - fugitive',
 'emission of pollutant gases - sulphur dioxide',
 'emission of pollutant gases - nitrogen oxides',
 'emission of pollutant gases - carbon monoxide',
 'emission

In [9]:
from sklearn.preprocessing import LabelEncoder
# encoding the other categorical columns
cat_cols = ['powiat_voivod', 'voivodship', 'season','holiday_name']
for col in cat_cols:
    le = LabelEncoder()
    train['powiat_voivod_encoded'] = le.fit_transform(train['powiat_voivod'])
    train['voivodship_encoded'] =le.fit_transform(train['voivodship'])
    train['season_encoded'] =le.fit_transform(train['season'])
    train['holiday_name_encoded'] =le.fit_transform(train['holiday_name'])

# TEST DATA

In [13]:
test.head()

,powiat_voivod,cloud_cover,wind_speed,humidity,precipitation,snow_depth,temperature,voivodship,animal stock - total,area by land - utilised agricultural area,...,"crop production - permanent meadows, area in ha, total","crop production - permanent pastures, area in ha, total",CAQI_level,isUrban,season,holiday_name,is_school_holiday,is_lockdown,is_have_coalplant,is_weekend
DATE,,,,,,,,,,,,,,,,,,,,,
2021-03-02,"powiat aleksandrowski, kujawsko-pomorskie",5.000000,0.0,92.5,0.0,0.0,2.7,kujawsko-pomorskie,39.393684,1.712729,...,3.642105,0.623158,low,0,Spring,Non Holiday,0.0,0.0,0.0,0
2021-03-02,"powiat augustowski, podlaskie",7.000000,60.0,90.0,0.0,0.0,2.3,podlaskie,51.660036,4.947356,...,20.197709,2.318264,vlow,0,Spring,Non Holiday,0.0,0.0,0.0,0
2021-03-02,"powiat bełchatowski, łódzkie",4.846046,0.0,85.0,0.0,0.0,3.7,łódzkie,42.485537,3.735109,...,7.089876,0.634298,low,0,Spring,Non Holiday,0.0,0.0,1.0,0
2021-03-02,"powiat biała podlaska, lubelskie",6.000000,0.0,89.0,0.0,0.0,3.7,lubelskie,19.326531,0.136890,...,7.204082,0.285714,medium,1,Spring,Non Holiday,0.0,0.0,0.0,0
2021-03-02,"powiat białystok, podlaskie",7.000000,0.0,91.5,0.0,0.0,2.5,podlaskie,51.666667,0.304191,...,20.196078,2.313725,low,1,Spring,Non Holiday,0.0,0.0,0.0,0


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60390 entries, 2021-03-02 to 2021-12-31
Columns: 123 entries, powiat_voivod to is_weekend
dtypes: float64(116), int32(1), int64(1), object(5)
memory usage: 56.9+ MB


In [15]:
test.shape

(60390, 123)

In [11]:
test['CAQI_level'].value_counts()

vlow      31913
low       24346
medium     3683
high        435
vhigh        13
Name: CAQI_level, dtype: int64

In [12]:
# combine high and vhigh to a single category, as we want to see how it will affect the accuracy of model
test['CAQI_level'] = test['CAQI_level'].replace({'vhigh':'high'})

In [13]:
test['CAQI_level'].value_counts()

vlow      31913
low       24346
medium     3683
high        448
Name: CAQI_level, dtype: int64

In [14]:
# order of mapping
mapping = {'vlow':0,'low':1,"medium":2,"high":3}
test['CAQI_level'] = test['CAQI_level'].replace(mapping)

In [15]:
test['CAQI_level'].value_counts()

0    31913
1    24346
2     3683
3      448
Name: CAQI_level, dtype: int64

In [16]:
# encoding the other categorical columns
from sklearn.preprocessing import LabelEncoder
cat_cols = ['powiat_voivod', 'voivodship', 'season','holiday_name']
for col in cat_cols:
    le = LabelEncoder()
    test['powiat_voivod_encoded'] = le.fit_transform(test['powiat_voivod'])
    test['voivodship_encoded'] = le.fit_transform(test['voivodship'])
    test['season_encoded'] = le.fit_transform(test['season'])
    test['holiday_name_encoded'] = le.fit_transform(test['holiday_name'])

In [17]:
# imputing train_test_split
X_train = train.drop(['CAQI_level','powiat_voivod', 'voivodship', 'season', 'holiday_name'], axis=1)
y_train = train['CAQI_level']
X_test = test.drop(['CAQI_level','powiat_voivod', 'voivodship', 'season', 'holiday_name'], axis=1)
y_test = test['CAQI_level']

In [18]:
#import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [19]:
clf = RandomForestClassifier(max_depth=2, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=42)

In [20]:
# Predict on the test set
y_pred = clf.predict(X_test)

# Accuracy score:
model_accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", model_accuracy)

Model Accuracy: 0.549511508527902


In [21]:
# Evaluating the model performance
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65     31913
           1       0.48      0.42      0.44     24346
           2       0.00      0.00      0.00      3683
           3       0.00      0.00      0.00       448

    accuracy                           0.55     60390
   macro avg       0.27      0.28      0.27     60390
weighted avg       0.50      0.55      0.52     60390



In [24]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(y_test, y_pred)

print(cm)

[[23080  8833     0     0]
 [14241 10105     0     0]
 [ 1844  1839     0     0]
 [  148   300     0     0]]


# cross validation

In [30]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


cv_scores = cross_val_score(clf, X_train, y_train, cv=kfold, scoring='accuracy')

# Cross-validation scores
print("Cross-validation scores:", cv_scores)
print("Mean CV accuracy:", cv_scores.mean())

Cross-validation scores: [0.49709264 0.49687666 0.49561404 0.49777379 0.49732523]
Mean CV accuracy: 0.49693646996278573
